<h1> Title: Sentiment Analysis</h1>

<strong>Overview: In this notebook, I aim to perform sentiment analysis</strong><br>
In this notebook, it covers:<br>
1.0 Emotion Prediction<br>
2.0 Sentiment Mapping<br>

In [14]:
import re
import pandas as pd
import requests

In [12]:
df = pd.read_csv('../Datasets/comments.csv',nrows=10)
df["textOriginal"]

0    The, uh, *shape* of the containers is somethin...
1          And with perfect people like you â¤ï¸ðŸŒ¸
2    Please don't call me sir😅😅 best part is you re...
3    Lol All you need is to put your hair in two po...
4                        It's "for the record" by Ooyy
5      All that to look just above average teanage boy
6                                              Alright
7       Please show us what these are like in the sun.
8                    This is a great point, thank you!
9    Pretty brave to try it anyway after dreaming t...
Name: textOriginal, dtype: object

## 1.0 Emotion Prediction

In [ ]:
API_URL = "http://localhost:8000/predict"
API_KEY = "my_secret_key_605"

def predict_emotion(comment: str, threshold: float = 0.8, use_gnn: bool = True):
    headers = {"x-api-key": API_KEY}
    payload = {
        "text": comment,
        "threshold": threshold,
        "use_gnn": use_gnn
    }
    response = requests.post(API_URL, json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()["predictions"]
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

In [33]:
df["emotion"] = df["textOriginal"].apply(predict_emotion)
print(df[["textOriginal", "emotion"]])

                                        textOriginal  \
0  The, uh, *shape* of the containers is somethin...   
1        And with perfect people like you â¤ï¸ðŸŒ¸   
2  Please don't call me sir😅😅 best part is you re...   
3  Lol All you need is to put your hair in two po...   
4                      It's "for the record" by Ooyy   
5    All that to look just above average teanage boy   
6                                            Alright   
7     Please show us what these are like in the sun.   
8                  This is a great point, thank you!   
9  Pretty brave to try it anyway after dreaming t...   

                                             emotion  
0  [[curiosity, 0.9928971529006958], [neutral, 0....  
1  [[admiration, 0.9870427846908569], [neutral, 0...  
2  [[neutral, 0.8885517120361328], [gratitude, 0....  
3  [[amusement, 0.9933191537857056], [curiosity, ...  
4                    [[neutral, 0.9633876085281372]]  
5  [[annoyance, 0.9610331058502197], [admiration,... 

## 2.0 Sentiment Mapping

In [16]:
# Emotion-to-sentiment mapping
emotion_to_sentiment = {
    "admiration": "positive",
    "amusement": "positive",
    "approval": "positive",
    "caring": "positive",
    "curiosity": "positive",
    "desire": "positive",
    "excitement": "positive",
    "gratitude": "positive",
    "joy": "positive",
    "love": "positive",
    "optimism": "positive",
    "pride": "positive",
    "relief": "positive",

    "anger": "negative",
    "annoyance": "negative",
    "confusion": "negative",
    "disappointment": "negative",
    "disapproval": "negative",
    "disgust": "negative",
    "embarrassment": "negative",
    "fear": "negative",
    "grief": "negative",
    "nervousness": "negative",
    "remorse": "negative",
    "sadness": "negative",

    "neutral": "neutral",
    "realization": "neutral",
    "surprise": "neutral"
}

In [ ]:
# Convert emotions → sentiment with scores
def map_to_sentiment(emotion_predictions):
    sentiment_scores = {"positive": 0.0, "negative": 0.0, "neutral": 0.0}
    for emotion, score in emotion_predictions:
        sentiment = emotion_to_sentiment.get(emotion, "neutral")
        sentiment_scores[sentiment] += score
    return sentiment_scores

# Get final sentiment (highest total score)
def get_final_sentiment(emotion_predictions):
    scores = map_to_sentiment(emotion_predictions)
    return max(scores, key=scores.get)

In [34]:
df["sentiment_scores"] = df["emotion"].apply(map_to_sentiment)
df["final_sentiment"] = df["emotion"].apply(get_final_sentiment)

print(df[["textOriginal", "emotion", "sentiment_scores", "final_sentiment"]])

                                        textOriginal  \
0  The, uh, *shape* of the containers is somethin...   
1        And with perfect people like you â¤ï¸ðŸŒ¸   
2  Please don't call me sir😅😅 best part is you re...   
3  Lol All you need is to put your hair in two po...   
4                      It's "for the record" by Ooyy   
5    All that to look just above average teanage boy   
6                                            Alright   
7     Please show us what these are like in the sun.   
8                  This is a great point, thank you!   
9  Pretty brave to try it anyway after dreaming t...   

                                             emotion  \
0  [[curiosity, 0.9928971529006958], [neutral, 0....   
1  [[admiration, 0.9870427846908569], [neutral, 0...   
2  [[neutral, 0.8885517120361328], [gratitude, 0....   
3  [[amusement, 0.9933191537857056], [curiosity, ...   
4                    [[neutral, 0.9633876085281372]]   
5  [[annoyance, 0.9610331058502197], [admiratio